In [33]:
import pandas as pd
import numpy as np

from RBO import *
from getRecommendations import *
from dataPrep import *
from sklearn.metrics import *
from scipy.stats import linregress

from sklearn.metrics.pairwise import *

import time

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [34]:
%%time
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
#prod_ing = prod_ing.sort_values(['id', 'order'])
df = dataPrep(ings, prods, prod_ing)

print('Number of products df: ', len(df))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products df:  15945
Number of unique ingredients:  6899
Number of total ingredients:  446618
CPU times: user 3.66 s, sys: 384 ms, total: 4.04 s
Wall time: 4.1 s


In [35]:
%%time
##Long dataframes
df2_A = pd.read_csv('data/prodIng_Wide_01.csv', index_col=0)
df2_B = pd.read_csv('data/prodIng_Wide_02.csv', index_col=0)
df2_C = pd.read_csv('data/prodIng_Wide_03.csv', index_col=0)
frames = [df2_A, df2_B, df2_C]
df2 = pd.concat(frames)
df2 = df2.T

CPU times: user 25.2 s, sys: 3.03 s, total: 28.2 s
Wall time: 30.4 s


# GET RECOMMENDATIONS


In [102]:
product ='Eau Frache Douceur Micellar Cleansing Water Face, Eyes, Lips'

##get indices of all products
indices = pd.Series(df.index, index=df['product'])#.drop_duplicates()
##find index of specified product
idx = indices[product]

print('Product:', df.iloc[idx]['product'])
print('Brand:', df.iloc[idx]['brand'])
print('Id:', idx)
print('Number of ingredients:', df.iloc[idx]['ingCount'])
print('Ingredients', df.iloc[idx]['ingList'])

Product: Eau Frache Douceur Micellar Cleansing Water Face, Eyes, Lips
Brand: Lancome
Id: 10285
Number of ingredients: 12
Ingredients ['Water', 'Hexylene Glycol', 'Glycerin', 'Poloxamer 184', 'Dihydrocholeth-30', 'Polyaminopropyl Biguanide', 'Benzyl Salicylate', 'Propylene Glycol', 'Fragrance', 'Disodium Cocoamphodiacetate', 'Disodium EDTA', 'Rosa Gallica Flower Extract']


## Run engine

In [103]:
###############################################################################################COSINE
t0 = time.time()
df_cos = df
column = 'product'

# Get the index of the product that matches the product name
df_cos['ing#List'] = df_cos['ing#List'].astype(str)
tfidf = TfidfVectorizer(binary = True, stop_words = None)
tfidf_matrix = tfidf.fit_transform(df_cos['ing#List'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Get the pairwsie similarity scores of all products with that product
sim_scores_all_cos = list(enumerate(cosine_sim[idx]))

# Sort the products based on the similarity scores
sim_scores_sorted_cos = sorted(sim_scores_all_cos, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores_ten_cos = sim_scores_sorted_cos[1:11]

# Get the product indices
product_indices_cos = [i[0] for i in sim_scores_ten_cos]

# Return the top 10 most similar products and their ingredients
recommended_cos = df[['product', 'brand','ingList']].iloc[product_indices_cos]

topTen_cos = pd.DataFrame(recommended_cos)
commonItems(product, topTen_cos, df)
topTen_cos.reset_index()

t1 = time.time()
total_cos = t1-t0
print('Cos time ', total_cos)

###############################################################################################RBO
t4 = time.time()
df_ROB = dataPrep(ings, prods, prod_ing)
##Set the rbo we want to use
rbo = rbo_min

itemLookup = df_ROB.loc[idx]['ing#List']
items = df_ROB['ing#List']
sim_ROB = [rbo(itemLookup,i, .9) for i in items] ##change method here

sim_scores_all_ROB = list(enumerate(sim_ROB))

# Sort the products based on the similarity scores
sim_scores_sorted_ROB = sorted(sim_scores_all_ROB, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores_ten_ROB = sim_scores_sorted_ROB[1:11]

# Get the product indices
product_indices_ROB = [i[0] for i in sim_scores_ten_ROB]

recommended_ROB = df[['product', 'brand','ingList']].iloc[product_indices_ROB]

topTen_ROB = pd.DataFrame(recommended_ROB)
commonItems(product, topTen_ROB, df)
topTen_ROB.reset_index()

t5 = time.time()
total_r = t5-t4
print('ROB_value time ', total_r)

###############################################################################################R
t2 = time.time()
#get linregress similarity and return r-value
sim_r = [linregress(df2[idx], df2[i])[2] for i in df2]

##collect all similarity scores (r_val)
sim_scores_all_r = list(enumerate(sim_r))

# Sort the products based on the similarity scores
sim_scores_sorted_r = sorted(sim_scores_all_r, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores_ten_r = sim_scores_sorted_r[1:11]

# Get the product indices
product_indices_r = [i[0] for i in sim_scores_ten_r]

recommended_r = df[['product', 'brand','ingList']].iloc[product_indices_r]

topTen_r = pd.DataFrame(recommended_r)
commonItems(product, topTen_r, df)
topTen_r.reset_index()

t3 = time.time()
total_r = t3-t2
print('R_value time ', total_r)

Cos time  26.9805428981781
ROB_value time  5.066394090652466
R_value time  35.58079481124878


In [109]:
print('Product:', df.iloc[4375]['product'])


Product: Purete Thermale Calming Cleansing Solution, for Sensitive Face and Eyes


In [104]:
topTen_cos

,product,brand,ingList,commonIng
10285,Eau Frache Douceur Micellar Cleansing Water Fa...,Lancome,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",12
4375,"Purete Thermale Calming Cleansing Solution, fo...",Vichy,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",11
744,Micellar Water Cleansing Solution,La Roche-Posay,"[Water, Hexylene Glycol, Poloxamer 184, Glycer...",9
15664,Micellar Water 3 In 1 (Moisturizing),L'Oreal Paris,"[Water, Hexylene Glycol, Glycerin, Rosa Gallic...",9
788,Micellar Cleansing Water Complete Cleanser Nor...,L'Oreal Paris,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",7
1975,Nutritioniste SkinActive Micellar Cleansing Wa...,Garnier,"[Water, Hexylene Glycol, Glycerin, Disodium Co...",7
693,Micellar Cleansing Water Complete Cleanser Nor...,L'Oreal Paris,"[Water, Hexylene Glycol, Poloxamer 184, Disodi...",6
2204,Pure Active Micellar Water Oily Skin,Garnier,"[Water, Hexylene Glycol, Disodium Cocoamphodia...",6
1370,Purete Thermale 3-In-1 One Step Micellar Water,Vichy,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",8
7186,Purete Thermale 3 in 1 Micellar Solution,Vichy Laboratories,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",8


In [105]:
topTen_ROB

,product,brand,ingList,commonIng
10285,Eau Frache Douceur Micellar Cleansing Water Fa...,Lancome,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",12
4375,"Purete Thermale Calming Cleansing Solution, fo...",Vichy,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",11
744,Micellar Water Cleansing Solution,La Roche-Posay,"[Water, Hexylene Glycol, Poloxamer 184, Glycer...",9
1370,Purete Thermale 3-In-1 One Step Micellar Water,Vichy,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",8
7186,Purete Thermale 3 in 1 Micellar Solution,Vichy Laboratories,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",8
15664,Micellar Water 3 In 1 (Moisturizing),L'Oreal Paris,"[Water, Hexylene Glycol, Glycerin, Rosa Gallic...",9
788,Micellar Cleansing Water Complete Cleanser Nor...,L'Oreal Paris,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",7
13007,Stripped Off Micellar Water,Nyx Cosmetics,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",7
15651,Micellar Water 3 in 1 Refreshing,L'Oreal Paris,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",7
1247,Skin Naturals Micellar Cleansing Water,Garnier,"[Water, Hexylene Glycol, Glycerin, Alcohol Den...",7


In [106]:
topTen_r

,product,brand,ingList,commonIng
10285,Eau Frache Douceur Micellar Cleansing Water Fa...,Lancome,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",12
4375,"Purete Thermale Calming Cleansing Solution, fo...",Vichy,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",11
744,Micellar Water Cleansing Solution,La Roche-Posay,"[Water, Hexylene Glycol, Poloxamer 184, Glycer...",9
15664,Micellar Water 3 In 1 (Moisturizing),L'Oreal Paris,"[Water, Hexylene Glycol, Glycerin, Rosa Gallic...",9
1370,Purete Thermale 3-In-1 One Step Micellar Water,Vichy,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",8
7186,Purete Thermale 3 in 1 Micellar Solution,Vichy Laboratories,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",8
788,Micellar Cleansing Water Complete Cleanser Nor...,L'Oreal Paris,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",7
1975,Nutritioniste SkinActive Micellar Cleansing Wa...,Garnier,"[Water, Hexylene Glycol, Glycerin, Disodium Co...",7
13007,Stripped Off Micellar Water,Nyx Cosmetics,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",7
15651,Micellar Water 3 in 1 Refreshing,L'Oreal Paris,"[Water, Hexylene Glycol, Glycerin, Poloxamer 1...",7


In [116]:
prod_search_id = 15664

sims_cos = [v for i, v in enumerate(sim_scores_all_cos) if v[0] == prod_search_id]
sims_ROB = [v for i, v in enumerate(sim_scores_all_ROB) if v[0] == prod_search_id]
sims_r = [v for i, v in enumerate(sim_scores_all_r) if v[0] == prod_search_id]

sim_df_cos = pd.DataFrame(sim_scores_all_cos, columns=['index', 'score']).sort_values(['score'], ascending = False).reset_index().drop(['level_0'], axis = 1)
sim_df_ROB = pd.DataFrame(sim_scores_all_ROB, columns=['index', 'score']).sort_values(['score'], ascending = False).reset_index().drop(['level_0'], axis = 1)
sim_df_r = pd.DataFrame(sim_scores_all_r, columns=['index', 'score']).sort_values(['score'], ascending = False).reset_index().drop(['level_0'], axis = 1)

index_return_cos = sim_df_cos[sim_df_cos['index'] == prod_search_id].index.item() 
index_return_rob = sim_df_rob[sim_df_ROB['index'] == prod_search_id].index.item() 
index_return_r = sim_df_r[sim_df_r['index'] == prod_search_id].index.item() 

list1 = df.loc[idx]['ingList']
list2 = df.loc[prod_search_id]['ingList']


print('---------------------------------------------------------------')
columns = ['id','product', 'brand', 'ings', '#ings', 'commIngs', 'scoreCos', 'rankCos', 'scoreRBO', 'rankRBO', 'scoreR', 'rankR']
index = [0]
df_results = pd.DataFrame(index=index, columns=columns)

##Overview
df_results['id'] = prod_search_id
df_results['product'] = df.iloc[prod_search_id]['product']
df_results['brand'] = df.iloc[prod_search_id]['brand']
df_results['#ings'] = df.iloc[prod_search_id]['ingCount']
df_results['commIngs'] = len(set(list1)&set(list2))

#Cosine
df_results['scoreCos'] = sims_cos[0][1]
df_results['rankCos'] = index_return_cos

#ROB
df_results['scoreRBO'] = sims_ROB[0][1]
df_results['rankRBO'] = index_return_rob

#R val
df_results['scoreR'] = sims_r[0][1]
df_results['rankR'] = index_return_r

print('Ingredients', df.iloc[prod_search_id]['ingList'])
print('---------------------------------------------------------------')

df_results

---------------------------------------------------------------
Ingredients ['Water', 'Hexylene Glycol', 'Glycerin', 'Rosa Gallica Flower Extract', 'Sorbitol', 'Poloxamer 184', 'Disodium Cocoamphodiacetate', 'Disodium EDTA', 'Propylene Glycol', 'BHT', 'Polyaminopropyl Biguanide']
---------------------------------------------------------------


,id,product,brand,ings,#ings,commIngs,scoreCos,rankCos,scoreRBO,rankRBO,scoreR,rankR
0,15664,Micellar Water 3 In 1 (Moisturizing),L'Oreal Paris,NaN,11,9,0.741894,4,0.669261,6,0.783028,4
